构建出源地址、目标地址、源经纬、目标经纬度的csv表格（不添加重复地址）

In [ ]:
import pandas as pd
from tqdm import tqdm

from itertools import permutations
df = pd.read_excel("data/CityList_333.xlsx")
lat_lon_df = pd.read_excel("data/CityList_333_经纬度_百度API.xlsx")

In [9]:
lat_lon_df.head(2)

,省份,地级行政区,经度,纬度
0,北京市,北京市,116.407001,39.904599
1,天津市,天津市,117.201561,39.085259


In [2]:
df.columns

Index(['省份', '地级行政区'], dtype='object')

In [3]:
df.head(2)

,省份,地级行政区
0,北京市,北京市
1,天津市,天津市


In [4]:
def get_row(row):
    return (row["省份"], row["地级行政区"])

data = df.apply(get_row, axis=1).to_list()

In [5]:
data[:3]

[('北京市', '北京市'), ('天津市', '天津市'), ('河北省', '石家庄市')]

In [7]:
len(data) ** 2

113569

In [6]:
# 自己到自己的位置的距离与费用是0
matrix = list(permutations(data, r=2))

In [10]:
len(matrix) + 333

113565

In [20]:
len(matrix)

113232

In [26]:
113232 / 2

56616.0

In [7]:
prov_city_unique = set()

for item in matrix:
    item_reverse = item[::-1]
    if item_reverse in prov_city_unique:
        continue
    else:
        prov_city_unique.add(item)

In [10]:
lat_lon_df.head(2)

,省份,地级行政区,经度,纬度
0,北京市,北京市,116.407001,39.904599
1,天津市,天津市,117.201561,39.085259


In [11]:
row[0]

('上海市', '上海市')

In [18]:
def filter_data(row, _df):
    prov, city = row
    result = _df[(_df["省份"] == prov) & (_df["地级行政区"] == city)]
    return result.iloc[0].to_dict()

In [19]:
filter_data(("湖北省", "武汉市"), lat_lon_df)

{'省份': '湖北省', '地级行政区': '武汉市', '经度': 114.3050004308504, '纬度': 30.59279957208177}

In [24]:
df_data = []
for row in tqdm(prov_city_unique):
    d = {}
    origin_d = filter_data(row[0], lat_lon_df)
    d["origin_prov"] = origin_d["省份"]
    d["origin_city"] = origin_d["地级行政区"]
    d["origin_经度"] = origin_d['经度']
    d["origin_纬度"] = origin_d['纬度']
    

    target_d = filter_data(row[1], lat_lon_df)
    d["target_prov"] = target_d["省份"]
    d["target_city"] = target_d["地级行政区"]
    d["target_经度"] = target_d['经度']
    d["target_纬度"] = target_d['纬度']
    
    df_data.append(d)

100%|██████████| 56616/56616 [00:18<00:00, 3042.89it/s]


In [27]:
len(df_data)

56616

In [28]:
pd.DataFrame(df_data).to_csv("data/路径规划_数据.csv", index=False)